In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
import numpy as np

model_path = "./final_amharic_ner_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForTokenClassification.from_pretrained(model_path)
model.eval()

def predict(text):
    tokens = tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        output = model(**tokens)
    logits = output.logits[0]
    probs = torch.nn.functional.softmax(logits, dim=-1)
    return probs.numpy(), tokens

In [ ]:
from lime.lime_text import LimeTextExplainer

class_names = list(model.config.id2label.values())
explainer = LimeTextExplainer(class_names=class_names)

def lime_wrapper(texts):
    probs, tokens = predict(texts[0])
    # Get the most confident tag per token
    token_probs = np.max(probs, axis=-1)
    return token_probs.reshape(1, -1)

# Pick a sample text
sample_text = "አማርኛ ሱቅ በአዲስ አበባ ነው"

# Run LIME
exp = explainer.explain_instance(sample_text, lime_wrapper, num_features=10)

# Visualize
exp.show_in_notebook()

In [ ]:
import shap

def shap_wrapper(texts):
    probs, tokens = predict(texts[0])
    return np.max(probs, axis=-1)

explainer = shap.Explainer(shap_wrapper, tokenizer)
shap_values = explainer([sample_text])

shap.plots.text(shap_values[0])